# Automated ML

## Overview

For this notebook we will be importing the dataset, creating an experiment, creating a compute cluster, cleanning and preprocessing the data before doing our train and test split, running the AutoML Experiment and selecting the best AutoML Model.

## Dependencies

First we will be importing all the needed dependencies to complete the project.

Warning: Install the `imblearn` library before running the rest of the notebook

In [1]:
pip install imblearn

  Using cached imblearn-0.0-py2.py3-none-any.whl (1.9 kB)
  Using cached imbalanced_learn-0.8.0-py3-none-any.whl (206 kB)
     |████████████████████████████████| 22.2 MB 6.4 MB/s eta 0:00:01
ERROR: raiwidgets 0.2.2 has requirement lightgbm>=3.1.1, but you'll have lightgbm 2.3.0 which is incompatible.
ERROR: azureml-train-automl-runtime 1.27.0 has requirement scikit-learn<0.23.0,>=0.19.0, but you'll have scikit-learn 0.24.2 which is incompatible.
ERROR: azureml-automl-runtime 1.27.0 has requirement scikit-learn<0.23.0,>=0.19.0, but you'll have scikit-learn 0.24.2 which is incompatible.
ERROR: autokeras 1.0.12 has requirement tensorflow>=2.3.0, but you'll have tensorflow 2.1.0 which is incompatible.
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade azureml-sdk[notebooks,contrib,automl] azureml-widgets

  Using cached azureml_sdk-1.28.0-py3-none-any.whl (4.4 kB)
  Using cached azureml_widgets-1.28.0-py3-none-any.whl (14.1 MB)
  Using cached azureml_dataset_runtime-1.28.0-py3-none-any.whl (3.5 kB)
  Using cached azureml_pipeline-1.28.0-py3-none-any.whl (3.7 kB)
  Using cached azureml_train_automl_client-1.28.0-py3-none-any.whl (120 kB)
  Using cached azureml_core-1.28.0-py3-none-any.whl (2.2 MB)
  Using cached azureml_train-1.28.0-py3-none-any.whl (3.3 kB)
  Using cached azureml_train_automl-1.28.0-py3-none-any.whl (3.4 kB)
  Using cached azureml_contrib_server-1.28.0-py3-none-any.whl (13 kB)
  Using cached azureml_contrib_services-1.28.0-py3-none-any.whl (5.8 kB)
  Using cached azureml_contrib_pipeline_steps-1.28.0-py3-none-any.whl (18 kB)
  Using cached azureml_contrib_notebook-1.28.0-py3-none-any.whl (29 kB)
  Using cached azureml_telemetry-1.28.0-py3-none-any.whl (30 kB)
  Using cached azureml_dataprep-2.15.0-py3-none-any.whl (39.4 MB)
  Using cached azureml_pipeline_steps-1.28.0-p

In [3]:
pip install azureml-train-automl-runtime

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install --upgrade scikit-learn

  Using cached scikit_learn-0.24.2-cp36-cp36m-manylinux2010_x86_64.whl (22.2 MB)
ERROR: raiwidgets 0.2.2 has requirement lightgbm>=3.1.1, but you'll have lightgbm 2.3.0 which is incompatible.
ERROR: azureml-train-automl-runtime 1.28.0 has requirement scikit-learn<0.23.0,>=0.19.0, but you'll have scikit-learn 0.24.2 which is incompatible.
ERROR: azureml-explain-model 1.27.0 has requirement azureml-interpret~=1.27.0, but you'll have azureml-interpret 1.28.0 which is incompatible.
ERROR: azureml-datadrift 1.27.0 has requirement azureml-core~=1.27.0, but you'll have azureml-core 1.28.0 which is incompatible.
ERROR: azureml-datadrift 1.27.0 has requirement azureml-dataset-runtime[fuse,pandas]~=1.27.0, but you'll have azureml-dataset-runtime 1.28.0 which is incompatible.
ERROR: azureml-datadrift 1.27.0 has requirement azureml-pipeline-core~=1.27.0, but you'll have azureml-pipeline-core 1.28.0 which is incompatible.
ERROR: azureml-datadrift 1.27.0 has requirement azureml-telemetry~=1.27.0, bu

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

from azureml.core.dataset import Dataset
from azureml.core import Datastore
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.train.automl import AutoMLConfig

from azureml.pipeline.steps import AutoMLStep

from azureml.data.dataset_factory import TabularDatasetFactory


# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.28.0


## Workspace Configuration

In this cell we import the workspace configuration and create an experiment that we will use later.

In [2]:
ws = Workspace.from_config()

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

# choose a name for experiment
experiment_name = 'automlcovid'

experiment=Experiment(ws, experiment_name)
experiment

Workspace name: quick-starts-ws-144486
Azure region: southcentralus
Subscription id: 1b944a9b-fdae-4f97-aeb1-b7eea0beac53
Resource group: aml-quickstarts-144486


Name,Workspace,Report Page,Docs Page
automatedmlcovid,quick-starts-ws-144486,Link to Azure Machine Learning studio,Link to Documentation


## Compute Cluster creation
In this cell a cpu cluster is created for running our experiments, it checks if a compute cluster with the same name exists, if it exists then uses it, if not it creates it.

If the cluster does not exists we define the configuration for it. For this project we will be using `min_nodes = 1`, in your own project this will incurr in extra costs, so consider leaving it to 0.

In [3]:
compute_cluster_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print("Found existing compute cluster...")
except:
    print("Creating new compute cluster...")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D16s_V3', max_nodes=4, min_nodes=1)
    compute_target = ComputeTarget.create(ws, compute_cluster_name, compute_config)
    
compute_target.wait_for_completion(show_output=True)
print("Cluster details: ", compute_target.get_status().serialize())

Found existing compute cluster...
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
Cluster details:  {'currentNodeCount': 4, 'targetNodeCount': 4, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 4, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-05-11T00:20:59.808000+00:00', 'errors': None, 'creationTime': '2021-05-10T20:19:57.705745+00:00', 'modifiedTime': '2021-05-10T20:20:13.081040+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D16S_V3'}


## Dataset

### Overview
We are going to use the Mexican Government's COVID-19 data, once it is uploaded to ML Studio we will consume it using TabularDataset.

To consume the dataset we will import it from the Datastore tab in ML Studio, this is a blob storage, then we have to specify the location of the csv to import it.

Once it's imported we will we creating a Pandas Dataframe that we will need later on.

In [4]:
datastore = Datastore.get(ws,'workspaceblobstore')
ds = TabularDatasetFactory.from_delimited_files(path=(datastore, 'UI/05-10-2021_081653_UTC/210509COVID19MEXICO.csv'))
df = ds.to_pandas_dataframe()

print(df)

        FECHA_ACTUALIZACION ID_REGISTRO  ORIGEN  SECTOR  ENTIDAD_UM  SEXO  \
0                2021-05-09      z482b8       1      12           9     2   
1                2021-05-09      z49a69       1      12          23     1   
2                2021-05-09      z23d9d       1      12          22     2   
3                2021-05-09      z24953       1      12           9     1   
4                2021-05-09      zz8e77       1      12           9     2   
...                     ...         ...     ...     ...         ...   ...   
6757355          2021-05-09      6bd21b       2      12           9     1   
6757356          2021-05-09      d0d932       2      12          12     1   
6757357          2021-05-09      7ceaf8       2       4           9     1   
6757358          2021-05-09      56cd1e       2       4           5     1   
6757359          2021-05-09      cd3ffd       2       4          15     2   

         ENTIDAD_NAC  ENTIDAD_RES  MUNICIPIO_RES  TIPO_PACIENTE  ...  \
0  

### Data Preprocessing

#### Cleaning
The objective of our project is to create a model that can get if a patient with COVID-19 will enter into an Intensive Care Unit or not.

The objective variable `UCI` has a lot of values that are 97, 98, or 99, which means that we don't know if the person is or was in a Intensive Care Unit, we only need the values 1 and 2, which means that the patient was or wasn't in an Intensive Care Unit, respectively.

So the next step is to trim our dataset by removing the rows that have 97, 98 or 99, leaving only 'Yes'/'No' for our objective variable.

In [5]:
arr_drop = [97,98,99]
for element in arr_drop:
    df.drop(df.loc[df['UCI']==element].index,inplace=True)

print(df)
print("--------------------------------------")
print("Classes count")
classes = pd.crosstab(index=df['UCI'],columns="count")
print(classes)


        FECHA_ACTUALIZACION ID_REGISTRO  ORIGEN  SECTOR  ENTIDAD_UM  SEXO  \
1                2021-05-09      z49a69       1      12          23     1   
16               2021-05-09      z166d5       1      12           1     1   
21               2021-05-09      z388cd       1      12          29     2   
22               2021-05-09      z4533d       1      12          29     1   
27               2021-05-09      z4494e       1      12           8     1   
...                     ...         ...     ...     ...         ...   ...   
6757282          2021-05-09      4b89bb       1      12          22     2   
6757292          2021-05-09      7efdce       1       4           9     2   
6757308          2021-05-09      d45a16       1       4          19     1   
6757320          2021-05-09      d3d30f       2       6          28     1   
6757347          2021-05-09      a7f5d8       1      12          18     1   

         ENTIDAD_NAC  ENTIDAD_RES  MUNICIPIO_RES  TIPO_PACIENTE  ...  \
1  

### Defining useful features
As we defined in our task definition in the `README.md` file, we will use only the following features: `SEXO`, `EDAD`, `NEUMONIA`, `DIABETES`, `EPOC`, `ASMA`, `INMUSUPR`, `HIPERTENSION`, `OTRA_COM`, `CARDIOVASCULAR`, `OBESIDAD`, `RENAL_CRONICA`, `TABAQUISMO`, `RESULTADO_LAB`.

In [6]:
df = df[['SEXO', 'EDAD','NEUMONIA','DIABETES','EPOC','ASMA','INMUSUPR','HIPERTENSION','OTRA_COM','CARDIOVASCULAR','OBESIDAD','RENAL_CRONICA','TABAQUISMO','RESULTADO_LAB', 'UCI']]
print(df)

         SEXO  EDAD  NEUMONIA  DIABETES  EPOC  ASMA  INMUSUPR  HIPERTENSION  \
1           1    66         1         1     2     2         2             1   
16          1    32         1         1     2     2         2             1   
21          2    12         1         1     2     2         2             2   
22          1    11         2         1     2     2         2             2   
27          1    49         2         2     2     2         2             2   
...       ...   ...       ...       ...   ...   ...       ...           ...   
6757282     2    87         1         2     2     2         2             2   
6757292     2    21         2         2     2     2         2             2   
6757308     1    69         2         2     2     2         2             2   
6757320     1    70         2         2     1     2         1             1   
6757347     1    33         1         2     2     2         2             2   

         OTRA_COM  CARDIOVASCULAR  OBESIDAD  RENAL_

#### Handling dataset class imbalance

As we can see, the rows with 1 (Yes) in the `UCI` column are a lot less than the 2 (No). we can send the dataset to training like this, but we will get a "Class balancing warning" alert in Azure ML, so we are going to fix it.

The next step in our data preprocessing is to balance the dataset. We will need the `imblearn` library that we installed before, and we will need to divide our dataframe in X and y to use the "RandomUnderSampler" method.

`y` has only the column `UCI` column, and `X` the rest of the columns, as we already defined the features we wanted in the previous cell

In [7]:
y = df['UCI']
X = df.drop('UCI',1)

rus = RandomUnderSampler(random_state = 42)
X_resampled, y_resampled = rus.fit_resample(X,y)
print(X_resampled)
print("-------")
print(y_resampled)

        SEXO  EDAD  NEUMONIA  DIABETES  EPOC  ASMA  INMUSUPR  HIPERTENSION  \
0          1    66         1         1     2     2         2             1   
1          2    60         1         1     2     2         2             2   
2          1    80         1         1     2     2         2             1   
3          1     5         1         2     2     2         2             2   
4          1    58         1         2     2     2         2             2   
...      ...   ...       ...       ...   ...   ...       ...           ...   
109779     2    63         2         2     2     2         2             2   
109780     2    10         2         2     2     2         2             2   
109781     2    45         2         2     2     2         2             2   
109782     2    53         1         2     2     2         2             1   
109783     2    50         1         2     2     2         2             2   

        OTRA_COM  CARDIOVASCULAR  OBESIDAD  RENAL_CRONICA  TABA

### Train/test split

Now that we have our balanced data, we will divide X and y between train and test

In [8]:
x_train, x_test, y_train, y_test = train_test_split(X_resampled,y_resampled,test_size = 0.2, random_state = 42)

train_data = pd.concat([x_train,y_train],axis=1)

print(train_data)

print("Validating that we have balanced data for training.")
classes = pd.crosstab(index=train_data['UCI'],columns="count")
print(classes)

        SEXO  EDAD  NEUMONIA  DIABETES  EPOC  ASMA  INMUSUPR  HIPERTENSION  \
29634      1    23         1         2     2     2         2             2   
106415     2    40         1         2     2     2         2             2   
78580      1    63         2         1     2     2         2             1   
45877      2    34         1         2     2     2         2             2   
524        2    33         1         2     2     2         2             2   
...      ...   ...       ...       ...   ...   ...       ...           ...   
54886      1    49         2         2     2     2         2             1   
76820      2    47         1         2     2     2         2             2   
103694     2    64         2         2     2     2         2             2   
860        2    46         1         2     2     2         2             2   
15795      2    32         2         2     2     2         2             2   

        OTRA_COM  CARDIOVASCULAR  OBESIDAD  RENAL_CRONICA  TABA

### Converting Train Data to Tabular Dataset
Since the train data is in the pandas Dataframe format, we have to convert it to Tabular Dataset to be used in AutoML

Warning: don't forget to drop the dataframe index, as the csv creates another index, and it will confuse our model.

In [9]:
#Convert train_data (Which are in pandas DataFrame format) to TabularDataset format.
try:
    os.makedirs('./data', exist_ok=True)
except OSError as error:
    print('New directory cannot be created')
    
path = 'data/traindata.csv'
train_data.to_csv(path, index= False)

datastore = ws.get_default_datastore()
datastore.upload(src_dir='data', target_path='data')

train_data = TabularDatasetFactory.from_delimited_files(path=[(datastore, ('data/traindata.csv'))])
print("Successfully converted the dataset to TabularDataset format.")
print(type(train_data))

Uploading an estimated of 1 files
Target already exists. Skipping upload for data/traindata.csv
Uploaded 0 files
Successfully converted the dataset to TabularDataset format.
<class 'azureml.data.tabular_dataset.TabularDataset'>


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

Since I was dealing with a large dataset, I needed to give more time to the experiment_timeout_minutes, that's why it's set to 1 hour.

In [10]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 60,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target=compute_target,
    task = "classification",
    training_data=train_data,
    label_column_name='UCI',   
    enable_early_stopping= True,
    featurization= 'auto',
    debug_log = "automl_errors.log",
    **automl_settings
)

## Submit the experiment with RunDetails widget
With the `RunDetails` widget we can see important outputs and when the run is completed.

In this cell we submit the experiment, with `show_output = True` to see the run logs in real time

In [11]:
from azureml.widgets import RunDetails
#Submit the experiment
remote_run = experiment.submit(automl_config)
#Show Run Details
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)


Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automatedmlcovid,AutoML_8d3c1dfb-f26f-45ac-a7b6-c0af7bbb4fef,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
automatedmlcovid,AutoML_8d3c1dfb-f26f-45ac-a7b6-c0af7bbb4fef,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.m

{'runId': 'AutoML_8d3c1dfb-f26f-45ac-a7b6-c0af7bbb4fef',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-05-11T00:27:19.601718Z',
 'endTimeUtc': '2021-05-11T00:49:17.079127Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"automatedmlcovid","subscription_id":"1b944a9b-fdae-4f97-aeb1-b7eea0beac53","resource_group":"aml-quickstarts-144486","workspace_name":"quick-starts-ws-144486","region":"southcentralus","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000,"primary_metric":"accuracy"

## Best Model

In the cell below we get the best model from the automl experiments and display all the properties of the model.



In [14]:
best_run_automl, fitted_model_automl = remote_run.get_output()

#Displaying properties of the AutoML Model
print("AutoML:")
automl_best_run_metrics = best_run_automl.get_metrics()
print("Best run Id: ",best_run_automl.id)
print("Accuracy: ", automl_best_run_metrics['accuracy'])
print("All metrics:")
for metric in automl_best_run_metrics:
    value = automl_best_run_metrics[metric]
    print(metric,value)

AutoML:
Best run Id:  AutoML_8d3c1dfb-f26f-45ac-a7b6-c0af7bbb4fef_40
Accuracy:  0.6459859174033216
All metrics:
recall_score_weighted 0.6459859174033216
AUC_weighted 0.689845848199903
f1_score_macro 0.6404632389588126
precision_score_micro 0.6459859174033216
recall_score_macro 0.6463376282851019
AUC_macro 0.6898458481949576
balanced_accuracy 0.6463376282851019
precision_score_weighted 0.6562473256050456
accuracy 0.6459859174033216
matthews_correlation 0.30227817574417126
log_loss 0.636956005701102
f1_score_micro 0.6459859174033216
recall_score_micro 0.6459859174033216
average_precision_score_weighted 0.6766319783075331
f1_score_weighted 0.6403549722166326
norm_macro_recall 0.29267525657020377
average_precision_score_micro 0.6825863157135152
weighted_accuracy 0.6456343591223622
average_precision_score_macro 0.6765515511633552
precision_score_macro 0.6560988374132936
AUC_micro 0.6955804682976401
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_8d3c1dfb-f26f-45ac-a7b6-c0af7bbb4fe

In [18]:
from pprint import pprint

# Function to list the hyperparameters 

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators' : list(e[0] for e in step[1].estimators), 'weights' : step[1].weights})
            print()

            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        
        else:
            pprint(step[1].get_params())
            print()
        
print_model(fitted_model_automl)

datatransformer
{'enable_dnn': False,
 'enable_feature_sweeping': True,
 'feature_sweeping_config': {},
 'feature_sweeping_timeout': 86400,
 'featurization_config': None,
 'force_text_dnn': False,
 'is_cross_validation': True,
 'is_onnx_compatible': False,
 'observer': None,
 'task': 'classification',
 'working_dir': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook144486/code/Users/odl_user_144486'}

prefittedsoftvotingclassifier
{'estimators': ['18', '0', '1', '6', '7', '35', '11', '15', '29', '21'],
 'weights': [0.23076923076923078,
             0.07692307692307693,
             0.15384615384615385,
             0.07692307692307693,
             0.07692307692307693,
             0.07692307692307693,
             0.07692307692307693,
             0.07692307692307693,
             0.07692307692307693,
             0.07692307692307693]}

18 - sparsenormalizer
{'copy': True, 'norm': 'l1'}

18 - xgboostclassifier
{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,


In [19]:
#Saving the best model
model_automl = best_run_automl.register_model(model_path='outputs/model.pkl', model_name='model_automl')
print("\nModel saved successfully")
print(model_automl)


Model saved successfully
Model(workspace=Workspace.create(name='quick-starts-ws-144486', subscription_id='1b944a9b-fdae-4f97-aeb1-b7eea0beac53', resource_group='aml-quickstarts-144486'), name=model_automl, id=model_automl:2, version=2, tags={}, properties={})


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

Before this step we have to run the `hyperparameter_tuning.ipynb`  notebook, in this case we determined that the model with the most accuracy was the one from AutoML. The next step is to deploy the model and test it.

First we have to download the scoring file and the environment file, this can be obtained from the best run.

In [20]:
# Download scoring file 
best_run_automl.download_file('outputs/scoring_file_v_1_0_0.py', 'scoring.py')

# Download environment file
best_run_automl.download_file('outputs/conda_env_v_1_0_0.yml', 'environment.yml')


Now we have to deploy the model with inference configuration

In [22]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

#Configuration for inference
inference_config = InferenceConfig(entry_script='scoring.py',
                                    environment=best_run_automl.get_environment())

#Deployment configuration
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, auth_enabled = True, enable_app_insights= True, description="Covid 19 ICU Prediction Model")
service = model_automl.deploy(ws, "icuservice", [model_automl], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-05-11 01:00:45+00:00 Creating Container Registry if not exists.
2021-05-11 01:00:45+00:00 Registering the environment.
2021-05-11 01:00:46+00:00 Use the existing image.
2021-05-11 01:00:46+00:00 Generating deployment configuration.
2021-05-11 01:00:47+00:00 Submitting deployment to compute..
2021-05-11 01:00:55+00:00 Checking the status of deployment icuservice..
2021-05-11 01:05:23+00:00 Checking the status of inference endpoint icuservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"


Here we check the state of the endpoint and some relevant data of it.

In [23]:
print("Service state:")
print(service.state)
print("Scoring URI:")
print(service.scoring_uri)
print("Swagger URI:")
print(service.swagger_uri)
print("Authentication key:")
print(service.get_keys()[0])

Service state:
Healthy
Scoring URI:
http://9311c453-9c2c-4e06-95bd-604f3cf53b7b.southcentralus.azurecontainer.io/score
Swagger URI:
http://9311c453-9c2c-4e06-95bd-604f3cf53b7b.southcentralus.azurecontainer.io/swagger.json
Authentication key:
nhJo7mSPydWh8MCQqNvMWpYTAoiVrMwI


In the cells below we send a request to the web service we deployed to test it.

In [24]:
import json

#Import test data
df = train_data.to_pandas_dataframe()
test_df = df.sample(5)
objective_df = test_df.pop('UCI')

test_sample = json.dumps({'data': test_df.to_dict(orient='records')})

print(test_sample)

{"data": [{"SEXO": 1, "EDAD": 57, "NEUMONIA": 1, "DIABETES": 1, "EPOC": 1, "ASMA": 2, "INMUSUPR": 2, "HIPERTENSION": 1, "OTRA_COM": 2, "CARDIOVASCULAR": 2, "OBESIDAD": 2, "RENAL_CRONICA": 1, "TABAQUISMO": 2, "RESULTADO_LAB": 2}, {"SEXO": 1, "EDAD": 69, "NEUMONIA": 1, "DIABETES": 2, "EPOC": 2, "ASMA": 2, "INMUSUPR": 2, "HIPERTENSION": 2, "OTRA_COM": 2, "CARDIOVASCULAR": 2, "OBESIDAD": 2, "RENAL_CRONICA": 2, "TABAQUISMO": 2, "RESULTADO_LAB": 1}, {"SEXO": 1, "EDAD": 91, "NEUMONIA": 1, "DIABETES": 2, "EPOC": 2, "ASMA": 2, "INMUSUPR": 2, "HIPERTENSION": 2, "OTRA_COM": 2, "CARDIOVASCULAR": 2, "OBESIDAD": 2, "RENAL_CRONICA": 2, "TABAQUISMO": 2, "RESULTADO_LAB": 2}, {"SEXO": 1, "EDAD": 85, "NEUMONIA": 2, "DIABETES": 2, "EPOC": 2, "ASMA": 2, "INMUSUPR": 2, "HIPERTENSION": 1, "OTRA_COM": 2, "CARDIOVASCULAR": 2, "OBESIDAD": 2, "RENAL_CRONICA": 1, "TABAQUISMO": 2, "RESULTADO_LAB": 97}, {"SEXO": 2, "EDAD": 21, "NEUMONIA": 1, "DIABETES": 2, "EPOC": 2, "ASMA": 2, "INMUSUPR": 2, "HIPERTENSION": 2, "OT

In [25]:
import requests

# Set the content type
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ service.get_keys()[0])}

response = requests.post(service.scoring_uri, test_sample, headers=headers)
print(response.text)
print(objective_df)

"{\"result\": [2, 1, 2, 2, 1]}"
2862     2
19451    1
79363    2
64269    2
42854    2
Name: UCI, dtype: int64


## ONIX Model

In [26]:
from azureml.automl.runtime.onnx_convert import OnnxConverter

automl_best_run_onnx, automl_fitted_model_onnx = remote_run.get_output(return_onnx_model=True)

OnnxConverter.save_onnx_model(automl_fitted_model_onnx, './output/best_automl_model.onnx')

OnnxConvertException: OnnxConvertException:
	Message: Requested an ONNX compatible model but the run has ONNX compatibility disabled.
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Requested an ONNX compatible model but the run has ONNX compatibility disabled.",
        "target": "onnx_compatible",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentInvalid"
            }
        }
    }
}

## Logs of the web service

In the cells below, we print the logs of the web service

In [27]:
print(service.get_logs())

2021-05-11T01:05:13,042549800+00:00 - gunicorn/run 
2021-05-11T01:05:13,051873800+00:00 - iot-server/run 
2021-05-11T01:05:13,064085200+00:00 - rsyslog/run 
2021-05-11T01:05:13,105365800+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_98cae94c606e3ceb655a787040a8a93c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_98cae94c606e3ceb655a787040a8a93c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_98cae94c606e3ceb655a787040a8a93c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_98cae94c606e3ceb655a787040a8a93c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_98cae94c606e3ceb655a787040a8a93c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

Finally, we delete the service

In [28]:
service.delete()